# SheetSimplify 

In this notebook, we shall implement a RAG LLM to retrieve data from an excel sheet. The excel sheet has been taken from <a href="https://www.kaggle.com/datasets/mragpavank/big-mart-sales-dataset">here</a> and focuses on sales data for some company. This sheet is meant to be an example for our prototype and can of course be replaced with excel sheets which are internal to the organization. 

## Data Preprocessing

In [44]:
import pandas as pd 

df = pd.read_csv("data/Train.csv")

In [45]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [47]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

A quick glance at the data shows us that there is no discrepancy in the data types and that there are couple of null entries in the columns "Item_Weight" and "Outlet_Size". We have to remember however, that this is a simple dataset downloaded from Kaggle and hence the data is extremely friendly. In real life, this is rarely the case and hence data pre-processing would take much more steps and much longer. 

## Building the LLM Agent  

Using LangChain, we shall now build an LLM that uses the DataFrame for its context. Under the hood, this would mean that the context is set to the particular sheet we provide and then a system prompt is generated to make sure that the LLM utilizes this context appropriately. 

The agent would also need to make API calls to OpenAI. Hence, we provide an OpenAI key to fulfill this requirement. 

In [48]:
import os 

# os.environ["OPENAI_API_KEY"] = "" # Your key goes here

In [49]:
from langchain_experimental.agents import create_csv_agent
from langchain_openai import ChatOpenAI

In [50]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) 
agent = create_csv_agent(llm, 
                         'data/Train.csv',
                         agent_type="openai-tools",  
                         verbose=True)

In [51]:
agent

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessage(content='\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nThis is the result of `print(df.head())`:\n|    | Item_Identifier   |   Item_Weight | Item_Fat_Content   |   Item_Visibility | Item_Type             |   Item_MRP | Outlet_Identifier   |   Outlet_Establishment_Year | Outlet_Size   | Outlet_Location_Type   | Outlet_Type       |   Item_Outlet

The agent we generated has the characteristics shown above. Note that we have not provided the LLM agent with any prompt. This is because we are using langchain's default function for this. 

We shall now try and test this model that we have built 

## Run the model!  

In [52]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape[0]'}`


8523There are 8523 rows in the dataframe.

> Finished chain.


'There are 8523 rows in the dataframe.'

In [53]:
agent.run("How many low fat items were sold in different categories?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Item_Fat_Content'] == 'Low Fat']['Item_Type'].value_counts()"}`


Item_Type
Household                840
Snack Foods              645
Fruits and Vegetables    580
Health and Hygiene       481
Frozen Foods             424
Dairy                    382
Soft Drinks              339
Canned                   314
Baking Goods             301
Hard Drinks              199
Meat                     159
Others                   156
Breads                   126
Starchy Foods             72
Breakfast                 39
Seafood                   32
Name: count, dtype: int64The number of low fat items sold in different categories are as follows:
- Household: 840
- Snack Foods: 645
- Fruits and Vegetables: 580
- Health and Hygiene: 481
- Frozen Foods: 424
- Dairy: 382
- Soft Drinks: 339
- Canned: 314
- Baking Goods: 301
- Hard Drinks: 199
- Meat: 159
- Others: 156
- Breads: 126
- Starchy Foods: 72
- Breakfast: 

'The number of low fat items sold in different categories are as follows:\n- Household: 840\n- Snack Foods: 645\n- Fruits and Vegetables: 580\n- Health and Hygiene: 481\n- Frozen Foods: 424\n- Dairy: 382\n- Soft Drinks: 339\n- Canned: 314\n- Baking Goods: 301\n- Hard Drinks: 199\n- Meat: 159\n- Others: 156\n- Breads: 126\n- Starchy Foods: 72\n- Breakfast: 39\n- Seafood: 32'

In [55]:
agent.run("What was the highest earning item category? How much were the earnings?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.groupby('Item_Type')['Item_Outlet_Sales'].sum().idxmax(), df.groupby('Item_Type')['Item_Outlet_Sales'].sum().max()"}`


('Fruits and Vegetables', 2820059.8168)The highest earning item category was "Fruits and Vegetables" with total earnings of $2,820,059.82.

> Finished chain.


'The highest earning item category was "Fruits and Vegetables" with total earnings of $2,820,059.82.'